<h1>Printing Optimization with Pyomo and ML</h1>

<h2>Add Imports</h2>

In [2]:
import json
import pyomo
import pandas as pd

<h2>Formatting the Data</h2>

In [11]:
data = pd.read_json('example.json')
data.head()

,name,model_name,uuid,materials,current_print,saved_models,status
0,Printer1,Prusa i3 MK3S+,a1b2c3d4-e5f6-7890-1234-567890abcdef,"[{'type': 'PLA', 'colors': ['red', 'blue', 'gr...","{'model_name': 'benchy.gcode', 'progress': 75,...","[benchy.gcode, storage_box.gcode, mini_figure....",printing
1,Printer2,Ender 3 V2,f0e9d8c7-b6a5-4321-fedc-ba9876543210,"[{'type': 'PLA', 'colors': ['white', 'grey']}]",None,"[calibration_cube.gcode, phone_stand.gcode]",idle
2,Printer3,Photon Mono X,1a2b3c4d-5e6f-7080-90a0-b0c0d0e0f0g0,"[{'type': 'Resin', 'colors': ['grey', 'clear']}]","{'model_name': 'resin_miniature.gcode', 'progr...","[resin_miniature.gcode, jewelry_mold.gcode]",completed
3,Printer4,LulzBot TAZ 6,abcdef01-2345-6789-abcd-ef0123456789,"[{'type': 'ABS', 'colors': ['black']}, {'type'...",None,[prototype_part.gcode],error


In [10]:
print_models = pd.read_json('models.json')
print_models.head()

,file_id,model_name,material,estimated_time,compatible_printers
0,1a2b3c4d-e5f6-7890-1234-567890abcdef,benchy.gcode,PLA,2025-05-20 02:30:00,"[Prusa i3 MK3S+, Creality Ender 3 V2]"
1,f0e9d8c7-b6a5-4321-fedc-ba9876543210,storage_box.gcode,PLA,2025-05-20 05:45:00,[Prusa i3 MK3S+]
2,c1d2e3f4-a5b6-9870-1234-567890abcdef,mini_figure.gcode,PETG,2025-05-20 01:15:00,[Prusa i3 MK3S+]
3,b6a5f0e9-d8c7-4321-fedc-ba9876543210,calibration_cube.gcode,PLA,2025-05-20 00:45:00,[Creality Ender 3 V2]
4,98765432-10ab-cdef-0123-456789abcdef,phone_stand.gcode,PLA,2025-05-20 01:00:00,[Creality Ender 3 V2]
